In [7]:
import pandas as pd

df = pd.read_csv('./datasets/hewiki/hebrew_nikud_dataset.csv')

In [8]:
# Display the first few rows of the dataframe
print(df.head())

# Show basic info about the dataframe
print(df.info())

# Show summary statistics for the length of text and words columns
df['text_length'] = df['text'].apply(len)
df['words_length'] = df['words'].apply(len)
print(df[['text_length', 'words_length']].describe())

# Check for missing values
print(df.isnull().sum())

# Display the distribution of nikud_mask lengths
df['nikud_mask_length'] = df['nikud_mask'].apply(lambda x: len(x.split()))
print(df['nikud_mask_length'].value_counts().head())

                                                text  \
0  מָתֵמָטִיקָה היא תחום דעת העוסק במושגים כגון כ...   
1  תחום מחקר נוסף הוא גודל, אשר מוביל לעוצמות ולת...   
2       ריגורוזיוּת היא מרכיב יסודי בכל הוכחה מתמטית   
3  חישובים בהיקף גדול קשים לאימות, ולכן יש הטועני...   
4  אַקְסיּוֹמָה, אמיתה, או הנחת יסוד (בכתיב ארכאי...   

                                               words  \
0  מָתֵמָטִיקָה היא תחום דעת העוסק במושגים כגון כ...   
1  תחום מחקר נוסף הוא גודל, אשר מוביל לעוצמות ולת...   
2       ריגורוזיוּת היא מרכיב יסודי בכל הוכחה מתמטית   
3  חישובים בהיקף גדול קשים לאימות, ולכן יש הטועני...   
4  אַקְסיּוֹמָה, אמיתה, או הנחת יסוד (בכתיב ארכאי...   

                                      nikud_mask  
0                          1 0 0 0 0 0 0 0 0 0 0  
1  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0  
2                                  1 0 0 0 0 0 0  
3                    0 0 0 0 0 0 0 0 0 0 0 0 1 0  
4            1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0  
<class 'pandas.core.

In [9]:
import re

NIKUD_PATTERN = re.compile(
    '['
    '\u05B0'  # sheva
    '\u05B1'  # hataf segol
    '\u05B2'  # hataf patah
    '\u05B3'  # hataf qamats
    '\u05B4'  # hiriq
    '\u05B5'  # tsere
    '\u05B6'  # segol
    '\u05B7'  # patah
    '\u05B8'  # qamats
    '\u05B9'  # holam
    '\u05BB'  # qubuts
    '\u05BC'  # dagesh or mapiq
    '\u05BD'  # meteg
    '\u05BF'  # rafe (rare)
    '\u05C1'  # shin dot
    '\u05C2'  # sin dot
    '\u05C7'  # qamats qatan
    ']'
)

In [10]:
# Check if each row contains at least one word with nikud using NIKUD_PATTERN
def has_word_with_nikud(text):
    return any(NIKUD_PATTERN.search(word) for word in text.split())

# Apply the function to the 'text' column and check if all rows satisfy the condition
all_have_nikud = df['text'].apply(has_word_with_nikud).all()
print("All rows contain at least one word with nikud:", all_have_nikud)

All rows contain at least one word with nikud: True
